# 1. K-nearest Neighbor Classifier
### 1.1. Model Definition
Non-parametric, no model, no learning.

- Given $X_{new}$, find its k-nearest neighbors according to some distance measure
    - Euclidean distance: $(X_i-X_{new})^T(X_i-X_{new})$
- Classify $X_{new}$ as the majority vote, based on the labels of these neighbors

# 2. Naïve Bayes Classifier
### 2.1. Model Definition
Model assumes all features are conditionally independent.

$$P(Y|X_1,\ldots,X_n)=\frac{P(Y)\prod_{i=1}^n P(X_i|Y)}{P(X_1,\dots,X_n)}$$


where 
- $X$ is a feature for a given sample. 
- $n$ is the total number of samples.
- $Y$ is the label for a given sample.


#### Subcomponents


$P(Y)=\pi^Y(1-\pi)^{1-Y}$

where 
- $\pi$ is a parameter defining $P(Y=1)$ and must be learned.

***
If $X_i$ is discrete, $P(X_i|Y)$ follows a Multinoulli distribution:

$P(X_i=k|Y=j)=\prod_k \theta_{ijk}^{(1-\delta(X_i,k))}$

where 
- $\delta(X_i,k)$ is an indicator function which takes value 1 if $X_i=k$ and value 0 otherwise.
- $k$: The number of possible values for a given feature.
***



If $X_i$ is continuous, $P(X_i|Y)$ follows a Gaussian distribution:

$P(X_i|Y)\sim N(\mu_i,\sigma_i^2)$

where 
- $\mu_i$ and $\sigma_i^2$ are parameters which must be learned. 
***
$P(X_1,\dots,X_n)$

where

$P(X_1,\dots,X_n)=P(Y=0)\sum_{i=1}^n P(X_i|Y=0)+P(Y=1)\sum_{i=1}^n P(X_i|Y=1)$

### 2.3. MLE for Naïve Bayes

$$\pi=\frac{s_1}{n}$$

where 
- $s_1$: number of samples with $Y=1$
- $n$: total number of samples

***

$$\theta_{ijk}=\frac{s_{1,k}}{s_1}$$

where
- $s_{1,k}$: number of samples with $X_i=k,Y=1$

In [2]:
# Given $X_{new}$:
# - Compute $P(Y=1)\prod_{i=1}^nP(X_j=X_{j_{new}}|Y=1)$
# - Compute $P(Y=0)\prod_{i=1}^nP(X_j=X_{j_{new}}|Y=0)$
# - Classify $X_{new}$ as the label with the higher probability.

### 2.4. MAP estimation for Naïve Bayes

$$\pi=\frac{s_1+\alpha_0}{n+\alpha_0 + \beta_0}$$

where 
- $s_1$: number of samples with $Y=1$
- $n$: total number of samples
- $\alpha_0$:
- $\beta_0$:

***

$$\theta_{ijk}=\frac{s_{1,k}+\alpha_{ijk0}}{s_1+\alpha_{ijk0}+\beta_{ijk0}}$$

where
- $s_{1,k}$: number of samples with $X_i=k,Y=1$
- $\alpha_{ijk0}$:
- $\beta_{ijk0}$:

# 3. Logistic Regression
### 3.1. Model Definition (two class)
Transforms continuous value from linear regression into discrete values for output.

$$P(Y=1|X=\{X_1,\ldots, X_n\})=\frac{1}{1+\exp{(w_0 + \sum_{i=1}^n w_iX_i)}}$$

$$P(Y=0|X=\{X_1,\ldots, X_n\})=\frac{\exp{(w_0 + \sum_{i=1}^n w_iX_i)}}{1+\exp{(w_0 + \sum_{i=1}^n w_iX_i)}}$$

### 3.2. MCLE for Logistic Regression (two class)
We want to use MCLE to learn the model parameters. MCLE cannot be solved in closed-form with respect to $W$.

$$\hat{W}_{MCLE}=\underset{W}{argmax} \prod_l P(Y^l|X^l,W)$$

where
- $l$: number of training examples

### 3.3. MCLE  for Logistic Regression (two class) with Gradient Descent
Parameters can be derived using gradient descent because logistic regression is concave. Start with a random initialization of parameters. Repeat until the change is less than $\epsilon$, that is, until $l(W)_t - l(W)_{t-1} < \epsilon$.

$$l(W)=\sum_l Y^l(w_0+\sum_i^n w_i X^l_i)-ln(1+exp(w_0+\sum_i^n w_i X^l_i))$$

$$w_i \leftarrow w_i + \eta \nabla(W) $$

where 
- $\eta$ is step size (learning rate)
- $X_i^l$: value of $X_i$ for the $l$th training example.
- $\nabla(W)$ is the gradient

***

#### Subcomponents

$\nabla(W)=\frac{\partial l(W)}{\partial w_i}=\sum_l X^l_i\left(Y^l-\hat{P}(Y^l=1|X^l,W)\right)$

where
- $Y^l-\hat{P}(Y^l=1|X^l,W)$ is the prediction error
- $\hat{P}(Y^l=1|X^l,W)=\frac{exp(w_0+\sum_iw_iX_i)}{1+exp(w_0+\sum_iw_iX_i)}$


According to Mitchell, we accommodate weight $w_0$ by assuming an imaginary $X_0=1$ for all $l$.
***


In [ ]:
class LogisticRegression:
    
    def __init__(self, eta, epsilon):
        self.eta = eta
        self.epsilon = epsilon

    def calc_prediction(X,y,weights):
        pass

eta,epsilon

    def gradient_descent(X,y,eta,epsilon,weights):
        pass

    def fit(X,y,):
        # generate random weights for each feature
        weights = np.array([])

        # perform gradient descent until convergence
        weights = gradient_descent(X,y,eta,epsilon,weights)

        return weights
    

### 3.4. MAP estimation  for Logistic Regression (two class) with Gradient Descent
Regularization term helps reduce overfitting, especially when training data is sparse.

$$l(W)=?$$

$$w_i \leftarrow w_i -\eta\lambda w_i + \eta \sum_l X^l_i\left(Y^l-\hat{P}(Y^l=1|X^l,W)\right) $$

where
- $\lambda$ is a regularization term, $\lambda=\frac{1}{2\sigma^2}$

### 3.5. Model Definition (multiclass)
Logistic Regression for more than two classes. Learn $R-1$ set of weights.

For $k<R$:

$$P(Y=y_k|X=\{X_1,\ldots, X_n\})=\frac{\exp{(w_{k,0} + \sum_{i=1}^n w_{k,i}X_i)}}{1+\sum_{j=1}^{R-1}\exp{(w_{j,0} + \sum_{i=1}^n w_{ji}X_i)}}$$

where
- $R$: number of classes

***

For $k=R$:

$$P(Y=y_R|X=\{X_1,\ldots, X_n\})=\frac{1}{1+\sum_{j=1}^{R-1}\exp{(w_{j,0} + \sum_{i=1}^n w_{ji}X_i)}}$$